# Vector Scalping Strategy - Module Usage Demonstration

This notebook demonstrates the comprehensive usage of the vector scalping trading strategy modules:

- **calculations**: Vector calculation functions for trading strategy
- **data_service**: Async data fetching service using tvkit and Polars
- **signals**: Signal generation logic for vector scalping strategy
- **realdata**: Real-time data functionality (integrated into data_service)

All code examples follow the project's architectural standards with type safety, async-first patterns, and comprehensive error handling.

## Setup and Imports

First, let's import all necessary modules and setup logging for our demonstrations.

In [ ]:
import logging
import time
import random
from datetime import datetime
from typing import List, Dict, Optional, Tuple

# Data analysis and visualization
import polars as pl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Project modules
from vector_scalping import (
    # Core classes
    DataService,
    SignalGenerator,
    VectorCalculations,

    # Configuration and models
    StrategyConfig,
    RiskManagement,
    OHLCVData,
    PriceVector,
    MomentumVector,
    CombinedVector,
    TradingSignal,
    DivergenceSignal,

    # Enums
    TimeFrame,
    SignalType
)

# Configure logging for demonstrations
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ All modules imported successfully")
print("📦 Available modules: calculations, data_service, signals, models")
print(f"🕐 Current time: {datetime.now()}")

ImportError: cannot import name 'DivergenceSignal' from 'vector_scalping' (/Users/sarat/Code/program-trading/strategies/vector-scalping/vector_scalping/__init__.py)

## 1. Data Service Module

The `data_service` module provides async data fetching capabilities using the tvkit library for real-time forex data from TradingView. It includes:

- **Real-time data fetching** from TradingView
- **Multi-timeframe support** (5-min, 15-min, etc.)
- **Polars DataFrame integration** for efficient data analysis
- **Fallback to mock data** when tvkit is unavailable
- **Technical indicators** calculation
- **Type safety** with Pydantic validation

In [ ]:
async def demonstrate_data_service() -> Tuple[DataService, Dict[TimeFrame, List[OHLCVData]]]:
    """
    Demonstrate DataService functionality with type annotations and async patterns.

    Returns:
        Tuple of (DataService instance, multi-timeframe data)
    """
    print("🔄 Demonstrating DataService Module")
    print("=" * 50)

    # Create risk management configuration with type safety
    risk_mgmt: RiskManagement = RiskManagement(
        symbol="EURUSD",
        pip_size=0.0001,  # 4-decimal pair
        take_profit_pips=20,
        stop_loss_pips=30,
        is_decimal_4=True
    )

    # Create strategy configuration
    config: StrategyConfig = StrategyConfig(
        symbol="EURUSD",
        exchange="FX_IDC",
        vector_period=5,
        percentile_lookback=100,
        signal_threshold=60.0,
        direction_threshold=0.3,
        tf5_weight=0.7,
        tf15_weight=0.3,
        tf5_direction_weight=0.6,
        tf15_direction_weight=0.4,
        risk_management=risk_mgmt
    )

    print(f"📋 Created configuration for {config.symbol}")
    print(f"   Vector period: {config.vector_period} candles")
    print(f"   Signal threshold: {config.signal_threshold}th percentile")

    # Initialize DataService with async context manager
    try:
        async with DataService(config) as data_service:
            print("✅ DataService initialized successfully")

            # Validate symbol format
            is_valid: bool = data_service.validate_symbol_format(config.symbol)
            print(f"🔍 Symbol validation for {config.symbol}: {is_valid}")

            # Fetch multi-timeframe data concurrently
            print("📊 Fetching multi-timeframe data...")
            start_time = time.time()

            market_data: Dict[TimeFrame, List[OHLCVData]] = await data_service.fetch_multi_timeframe_data(
                bars_count=50
            )

            fetch_time = time.time() - start_time
            print(f"⏱️  Data fetched in {fetch_time:.2f} seconds")

            # Display data summary
            tf5_data: List[OHLCVData] = market_data[TimeFrame.MIN_5]
            tf15_data: List[OHLCVData] = market_data[TimeFrame.MIN_15]

            print(f"📈 5-min data: {len(tf5_data)} bars")
            print(f"📈 15-min data: {len(tf15_data)} bars")

            if tf5_data:
                latest: OHLCVData = tf5_data[-1]
                print(f"💰 Latest price: {latest.close:.5f} (Volume: {latest.volume:.0f})")
                print(f"🕐 Latest timestamp: {latest.datetime}")

            return data_service, market_data

    except Exception as e:
        print(f"❌ Error in DataService demonstration: {e}")
        raise

# Run the demonstration
data_service, market_data = await demonstrate_data_service()

NameError: name 'TimeFrame' is not defined

### Error Handling Example for DataService

The DataService includes comprehensive error handling with fallback mechanisms:

In [ ]:
async def demonstrate_data_service_error_handling() -> None:
    """
    Demonstrate error handling patterns in DataService.
    """
    print("🛡️ Demonstrating DataService Error Handling")
    print("=" * 50)

    # Example 1: Invalid symbol validation
    try:
        risk_mgmt = RiskManagement(
            symbol="INVALID",
            pip_size=0.0001,
            take_profit_pips=20,
            stop_loss_pips=30
        )
        config = StrategyConfig(
            symbol="INVALID",  # Invalid symbol format
            exchange="FX_IDC",
            risk_management=risk_mgmt
        )

        async with DataService(config) as service:
            is_valid = service.validate_symbol_format(config.symbol)
            print(f"❌ Invalid symbol '{config.symbol}' validation: {is_valid}")

            if not is_valid:
                print("⚠️  Symbol format validation failed - would prevent trading")

    except Exception as e:
        print(f"🔍 Caught validation error: {type(e).__name__}: {e}")

    # Example 2: Handling data fetch errors with fallback
    try:
        print("\n📡 Testing fallback to mock data...")

        # The DataService automatically falls back to mock data if tvkit fails
        # This ensures the notebook remains functional even without internet
        risk_mgmt = RiskManagement(
            symbol="EURUSD",
            pip_size=0.0001,
            take_profit_pips=20,
            stop_loss_pips=30
        )
        config = StrategyConfig(
            symbol="EURUSD",
            exchange="FX_IDC",
            risk_management=risk_mgmt
        )

        async with DataService(config) as service:
            # This will use real data if available, mock data if not
            data = await service.fetch_historical_data(TimeFrame.MIN_5, bars_count=10)
            print(f"📊 Successfully fetched {len(data)} bars (real or mock data)")

            if data:
                print(f"💡 Data type verification: {type(data[0])}")
                print("✅ All data validated through Pydantic models")

    except Exception as e:
        print(f"🔍 Caught data fetch error: {type(e).__name__}: {e}")

    print("\n🎯 Error handling demonstration completed")

await demonstrate_data_service_error_handling()

### Polars DataFrame Integration

The DataService provides seamless integration with Polars for efficient data analysis:

In [ ]:
def demonstrate_polars_integration(data_service: DataService, ohlcv_data: List[OHLCVData]) -> pl.DataFrame:
    """
    Demonstrate Polars DataFrame integration for data analysis.

    Args:
        data_service: DataService instance
        ohlcv_data: OHLCV data to analyze

    Returns:
        Polars DataFrame with technical indicators
    """
    print("🐻‍❄️ Demonstrating Polars DataFrame Integration")
    print("=" * 50)

    # Convert OHLCV data to Polars DataFrame
    df: pl.DataFrame = data_service.convert_to_polars_dataframe(ohlcv_data)
    print(f"📊 Created DataFrame with shape: {df.shape}")
    print(f"🔧 Columns: {df.columns}")

    # Add technical indicators
    df_with_indicators: pl.DataFrame = data_service.add_technical_indicators(df)
    print("📈 Added technical indicators")
    print(f"🔧 Updated columns: {df_with_indicators.columns}")

    # Display basic statistics
    if not df_with_indicators.is_empty():
        stats = df_with_indicators.select([
            pl.col("close").mean().alias("avg_close"),
            pl.col("close").std().alias("close_std"),
            pl.col("volume").mean().alias("avg_volume"),
            pl.col("atr_5").mean().alias("avg_atr_5"),
            pl.col("volatility_5").mean().alias("avg_volatility_5")
        ]).row(0, named=True)

        print("\n📊 Data Statistics:")
        print(f"   Average close: {stats['avg_close']:.5f}")
        print(f"   Close std dev: {stats['close_std']:.5f}")
        print(f"   Average volume: {stats['avg_volume']:.0f}")

        atr_val = stats['avg_atr_5']
        vol_val = stats['avg_volatility_5']
        print(f"   Average ATR(5): {f'{atr_val:.5f}' if atr_val is not None else 'N/A'}")
        print(f"   Average Volatility(5): {f'{vol_val:.5f}' if vol_val is not None else 'N/A'}")

    # Show sample of latest data
    print("\n📋 Latest 3 bars with indicators:")
    latest_data = df_with_indicators.select([
        "timestamp", "close", "sma_5", "sma_20", "atr_5", "returns"
    ]).tail(3)

    for row in latest_data.iter_rows(named=True):
        ts = datetime.fromtimestamp(row['timestamp'])
        close = row['close']
        sma_5 = row['sma_5']
        sma_20 = row['sma_20']
        atr_5 = row['atr_5']
        returns = row['returns']

        print(f"   {ts.strftime('%Y-%m-%d %H:%M')}: Close={close:.5f}, SMA5={f'{sma_5:.5f}' if sma_5 else 'N/A'}, "
              f"SMA20={f'{sma_20:.5f}' if sma_20 else 'N/A'}, ATR5={f'{atr_5:.5f}' if atr_5 else 'N/A'}")

    return df_with_indicators

# Extract 5-minute data for demonstration
tf5_data = market_data[TimeFrame.MIN_5]
df_with_indicators = demonstrate_polars_integration(data_service, tf5_data)

NameError: name 'market_data' is not defined

## 2. Calculations Module

The `calculations` module implements the core vector mathematics for the trading strategy:

- **Price Vector Calculation**: Displacement, magnitude, and direction over N candles
- **Momentum Vector Calculation**: Price momentum combined with volatility (ATR)
- **Multi-timeframe Vector Combination**: Weighted combination of 5-min and 15-min vectors
- **Percentile Ranking**: Signal strength calculation
- **Divergence Detection**: Price vs momentum divergence analysis

In [ ]:
def demonstrate_vector_calculations(
    tf5_data: List[OHLCVData],
    tf15_data: List[OHLCVData]
) -> Tuple[PriceVector, MomentumVector, CombinedVector]:
    """
    Demonstrate vector calculation functionality with type annotations.

    Args:
        tf5_data: 5-minute OHLCV data
        tf15_data: 15-minute OHLCV data

    Returns:
        Tuple of (price_vector, momentum_vector, combined_vector)
    """
    print("🧮 Demonstrating Vector Calculations Module")
    print("=" * 50)

    vector_period: int = 5

    # 1. Price Vector Calculation for 5-minute data
    print("\n📊 5-Minute Price Vector Calculation:")
    tf5_price: PriceVector = VectorCalculations.calculate_price_vector(
        data=tf5_data,
        period=vector_period
    )

    print(f"   Displacement: {tf5_price.displacement:.5f}")
    print(f"   Magnitude: {tf5_price.magnitude:.5f}")
    print(f"   Direction: {tf5_price.direction:.3f} (range: -1 to 1)")
    print(f"   Start price: {tf5_price.start_price:.5f}")
    print(f"   End price: {tf5_price.end_price:.5f}")
    print(f"   Price range: {tf5_price.price_range:.5f}")

    # 2. Momentum Vector Calculation for 5-minute data
    print("\n🚀 5-Minute Momentum Vector Calculation:")
    tf5_momentum: MomentumVector = VectorCalculations.calculate_momentum_vector(
        data=tf5_data,
        period=vector_period
    )

    print(f"   Price momentum: {tf5_momentum.price_momentum:.5f}")
    print(f"   Volatility (ATR): {tf5_momentum.volatility:.5f}")
    print(f"   Magnitude: {tf5_momentum.magnitude:.5f}")
    print(f"   Direction: {tf5_momentum.direction:.3f} (range: -1 to 1)")

    # 3. Same calculations for 15-minute data
    print("\n📊 15-Minute Vector Calculations:")
    tf15_price: PriceVector = VectorCalculations.calculate_price_vector(
        data=tf15_data,
        period=vector_period
    )
    tf15_momentum: MomentumVector = VectorCalculations.calculate_momentum_vector(
        data=tf15_data,
        period=vector_period
    )

    print(f"   Price direction: {tf15_price.direction:.3f}")
    print(f"   Momentum direction: {tf15_momentum.direction:.3f}")
    print(f"   Momentum magnitude: {tf15_momentum.magnitude:.5f}")

    # 4. Multi-timeframe Vector Combination
    print("\n🔄 Multi-timeframe Vector Combination:")
    combined: CombinedVector = VectorCalculations.combine_vectors(
        tf5_price=tf5_price,
        tf5_momentum=tf5_momentum,
        tf15_price=tf15_price,
        tf15_momentum=tf15_momentum,
        tf5_weight=0.7,
        tf15_weight=0.3,
        tf5_dir_weight=0.6,
        tf15_dir_weight=0.4
    )

    print(f"   Combined magnitude: {combined.combined_magnitude:.5f}")
    print(f"   Combined direction: {combined.combined_direction:.3f}")
    print(f"   5m contribution: magnitude={combined.tf5_magnitude:.5f}, direction={combined.tf5_direction:.3f}")
    print(f"   15m contribution: magnitude={combined.tf15_magnitude:.5f}, direction={combined.tf15_direction:.3f}")

    # 5. Percentile Ranking Example
    print("\n📈 Percentile Ranking Calculation:")

    # Create some historical magnitude data for demonstration
    historical_magnitudes: List[float] = [
        combined.combined_magnitude * random.uniform(0.5, 1.5)
        for _ in range(20)
    ]

    percentile_rank: float = VectorCalculations.calculate_percentile_rank(
        current_value=combined.combined_magnitude,
        historical_values=historical_magnitudes
    )

    print(f"   Current magnitude: {combined.combined_magnitude:.5f}")
    print(f"   Historical data points: {len(historical_magnitudes)}")
    print(f"   Percentile rank: {percentile_rank:.1f}th percentile")

    return tf5_price, tf5_momentum, combined

# Run vector calculations demonstration
tf5_data = market_data[TimeFrame.MIN_5]
tf15_data = market_data[TimeFrame.MIN_15]

price_vector, momentum_vector, combined_vector = demonstrate_vector_calculations(tf5_data, tf15_data)

### Divergence Detection

The calculations module includes divergence detection between price and momentum:

In [ ]:
def demonstrate_divergence_detection(tf5_data: List[OHLCVData]) -> Optional[DivergenceSignal]:
    """
    Demonstrate divergence detection functionality.

    Args:
        tf5_data: 5-minute OHLCV data

    Returns:
        DivergenceSignal if sufficient data, None otherwise
    """
    print("🔄 Demonstrating Divergence Detection")
    print("=" * 50)

    vector_period: int = 5
    required_length: int = vector_period * 2

    if len(tf5_data) < required_length:
        print(f"⚠️  Insufficient data for divergence detection (need {required_length}, have {len(tf5_data)})")
        return None

    # Split data into current and previous periods
    current_period: List[OHLCVData] = tf5_data[-vector_period:]
    previous_period: List[OHLCVData] = tf5_data[-(vector_period * 2):-vector_period]

    print(f"📊 Analyzing divergence between two {vector_period}-candle periods")
    print(f"   Previous period: {previous_period[0].datetime} to {previous_period[-1].datetime}")
    print(f"   Current period: {current_period[0].datetime} to {current_period[-1].datetime}")

    try:
        divergence: DivergenceSignal = VectorCalculations.detect_divergence(
            current_data=current_period,
            comparison_data=previous_period,
            period=vector_period
        )

        print("\n🔍 Divergence Analysis Results:")
        print(f"   Price trend: {divergence.price_trend:.4f} ({'📈' if divergence.price_trend > 0 else '📉'})")
        print(f"   Momentum trend: {divergence.momentum_trend:.4f} ({'📈' if divergence.momentum_trend > 0 else '📉'})")
        print(f"   Bullish divergence: {divergence.is_bullish_divergence} 🟢")
        print(f"   Bearish divergence: {divergence.is_bearish_divergence} 🔴")
        print(f"   Divergence strength: {divergence.divergence_strength:.3f}")

        if divergence.is_bullish_divergence:
            print("   🎯 Bullish divergence detected: Price declining while momentum increasing")
        elif divergence.is_bearish_divergence:
            print("   🎯 Bearish divergence detected: Price rising while momentum declining")
        else:
            print("   ✅ No significant divergence detected")

        return divergence

    except ValueError as e:
        print(f"❌ Divergence detection error: {e}")
        return None

# Demonstrate divergence detection
divergence_signal = demonstrate_divergence_detection(tf5_data)

### Error Handling in Calculations

The calculations module includes comprehensive validation and error handling:

In [ ]:
def demonstrate_calculations_error_handling() -> None:
    """
    Demonstrate error handling in vector calculations.
    """
    print("🛡️ Demonstrating Calculations Error Handling")
    print("=" * 50)

    # Example 1: Insufficient data
    try:
        insufficient_data: List[OHLCVData] = tf5_data[:2]  # Only 2 candles
        VectorCalculations.calculate_price_vector(insufficient_data, period=5)
    except ValueError as e:
        print(f"✅ Caught insufficient data error: {e}")

    # Example 2: Invalid period
    try:
        VectorCalculations.calculate_price_vector(tf5_data, period=0)
    except ValueError as e:
        print(f"✅ Caught invalid period error: {e}")

    # Example 3: Invalid weight combinations
    try:
        # Create dummy vectors for testing
        dummy_price = PriceVector(
            displacement=0.001,
            magnitude=0.001,
            direction=0.5,
            period=5,
            start_price=1.0850,
            end_price=1.0851,
            price_range=0.002
        )
        dummy_momentum = MomentumVector(
            price_momentum=0.0002,
            volatility=0.0005,
            magnitude=0.0006,
            direction=0.3,
            period=5
        )

        VectorCalculations.combine_vectors(
            tf5_price=dummy_price,
            tf5_momentum=dummy_momentum,
            tf15_price=dummy_price,
            tf15_momentum=dummy_momentum,
            tf5_weight=0.8,  # Invalid: 0.8 + 0.3 != 1.0
            tf15_weight=0.3
        )
    except ValueError as e:
        print(f"✅ Caught invalid weights error: {e}")

    # Example 4: Empty historical data for percentile
    try:
        VectorCalculations.calculate_percentile_rank(0.5, [])
    except ValueError as e:
        print(f"✅ Caught empty historical data error: {e}")

    print("\n🎯 All error handling tests passed")

demonstrate_calculations_error_handling()

## 3. Signals Module

The `signals` module generates trading signals based on vector analysis:

- **Signal Generation**: LONG/SHORT entry conditions based on vector analysis
- **Confidence Scoring**: Multi-factor confidence calculation
- **Risk Management Integration**: Automatic take profit and stop loss calculation
- **Time-based Exits**: Friday 5 PM GMT exit logic
- **Historical Data Management**: Percentile ranking with rolling windows

In [ ]:
async def demonstrate_signal_generation(
    tf5_data: List[OHLCVData],
    tf15_data: List[OHLCVData]
) -> TradingSignal:
    """
    Demonstrate signal generation functionality with async patterns.

    Args:
        tf5_data: 5-minute OHLCV data
        tf15_data: 15-minute OHLCV data

    Returns:
        Generated trading signal
    """
    print("📊 Demonstrating Signal Generation Module")
    print("=" * 50)

    # Create configuration for signal generation
    risk_mgmt: RiskManagement = RiskManagement(
        symbol="EURUSD",
        pip_size=0.0001,
        take_profit_pips=20,
        stop_loss_pips=30,
        is_decimal_4=True
    )

    config: StrategyConfig = StrategyConfig(
        symbol="EURUSD",
        exchange="FX_IDC",
        vector_period=5,
        percentile_lookback=100,
        signal_threshold=60.0,
        direction_threshold=0.3,
        tf5_weight=0.7,
        tf15_weight=0.3,
        tf5_direction_weight=0.6,
        tf15_direction_weight=0.4,
        risk_management=risk_mgmt
    )

    # Initialize signal generator
    signal_generator: SignalGenerator = SignalGenerator(config)
    print(f"🎯 Created SignalGenerator with {config.signal_threshold}th percentile threshold")

    # Build historical data for percentile calculation
    print("\n📈 Building historical magnitude data...")
    for i in range(20):
        if len(tf5_data) > i + config.vector_period and len(tf15_data) > i + config.vector_period:
            end_idx = -i if i > 0 else None
            window_5m = tf5_data[-(config.vector_period + i):end_idx]
            window_15m = tf15_data[-(config.vector_period + i):end_idx]

            if len(window_5m) >= config.vector_period and len(window_15m) >= config.vector_period:
                current_price = window_5m[-1].close
                temp_signal = signal_generator.generate_signal(window_5m, window_15m, current_price)

    historical_count = len(signal_generator.historical_magnitudes)
    print(f"📊 Built {historical_count} historical magnitude values")

    # Generate current signal
    current_price: float = tf5_data[-1].close
    print(f"💰 Current price: {current_price:.5f}")

    signal: TradingSignal = signal_generator.generate_signal(
        tf5_data=tf5_data,
        tf15_data=tf15_data,
        current_price=current_price
    )

    # Display signal analysis
    print("\n🎯 Signal Analysis Results:")
    print(f"   Signal Type: {signal.signal_type.value} {'🟢' if signal.signal_type == SignalType.LONG else '🔴' if signal.signal_type == SignalType.SHORT else '⚪'}")
    print(f"   Confidence: {signal.confidence:.2f} ({'High' if signal.confidence > 0.7 else 'Medium' if signal.confidence > 0.5 else 'Low'})")
    print(f"   Timestamp: {signal.datetime}")
    print(f"   Reason: {signal.reason}")

    # Display vector data if available
    if signal.vector_data:
        vector_data: CombinedVector = signal.vector_data
        print("\n📊 Vector Analysis:")
        print(f"   Combined magnitude: {vector_data.combined_magnitude:.5f}")
        print(f"   Combined direction: {vector_data.combined_direction:.3f}")
        print(f"   Signal strength: {vector_data.signal_strength:.1f}th percentile")
        print(f"   5m direction: {vector_data.tf5_direction:.3f}")
        print(f"   15m direction: {vector_data.tf15_direction:.3f}")

        # Interpretation
        if vector_data.combined_direction > 0.3:
            print("   🟢 Strong bullish bias detected")
        elif vector_data.combined_direction < -0.3:
            print("   🔴 Strong bearish bias detected")
        else:
            print("   🟡 Neutral/weak directional bias")

    # Display divergence data if available
    if signal.divergence_data:
        div_data: DivergenceSignal = signal.divergence_data
        print("\n🔄 Divergence Analysis:")
        print(f"   Price trend: {div_data.price_trend:.4f}")
        print(f"   Momentum trend: {div_data.momentum_trend:.4f}")
        print(f"   Bullish divergence: {div_data.is_bullish_divergence}")
        print(f"   Bearish divergence: {div_data.is_bearish_divergence}")
        print(f"   Divergence strength: {div_data.divergence_strength:.3f}")

    # Display trade setup if signal is actionable
    if signal.signal_type != SignalType.NO_SIGNAL and signal.entry_price:
        print("\n💼 Trade Setup:")
        print(f"   Entry price: {signal.entry_price:.5f}")
        print(f"   Take profit: {signal.take_profit:.5f}")
        print(f"   Stop loss: {signal.stop_loss:.5f}")

        # Calculate pip values
        if signal.signal_type == SignalType.LONG:
            tp_pips = (signal.take_profit - signal.entry_price) / config.risk_management.pip_size
            sl_pips = (signal.entry_price - signal.stop_loss) / config.risk_management.pip_size
        else:
            tp_pips = (signal.entry_price - signal.take_profit) / config.risk_management.pip_size
            sl_pips = (signal.stop_loss - signal.entry_price) / config.risk_management.pip_size

        print(f"   Take profit: {tp_pips:.0f} pips")
        print(f"   Stop loss: {sl_pips:.0f} pips")
        print(f"   Risk/Reward ratio: 1:{tp_pips/sl_pips:.2f}")

    # Check time-based exit condition
    should_exit: bool = signal_generator.should_exit_time_based()
    print(f"\n⏰ Time-based exit check (Friday 5 PM GMT): {should_exit}")

    return signal

# Run signal generation demonstration
trading_signal = await demonstrate_signal_generation(tf5_data, tf15_data)

### Signal Generation Error Handling

The signals module includes robust error handling for various edge cases:

In [ ]:
def demonstrate_signals_error_handling() -> None:
    """
    Demonstrate error handling in signal generation.
    """
    print("🛡️ Demonstrating Signals Error Handling")
    print("=" * 50)

    # Create minimal configuration
    risk_mgmt = RiskManagement(
        symbol="EURUSD",
        pip_size=0.0001,
        take_profit_pips=20,
        stop_loss_pips=30
    )
    config = StrategyConfig(
        symbol="EURUSD",
        exchange="FX_IDC",
        risk_management=risk_mgmt
    )

    signal_generator = SignalGenerator(config)

    # Example 1: Insufficient 5-minute data
    try:
        insufficient_5m = tf5_data[:3]  # Only 3 candles, need 5
        signal_generator.generate_signal(insufficient_5m, tf15_data, 1.0850)
    except ValueError as e:
        print(f"✅ Caught insufficient 5m data error: {e}")

    # Example 2: Insufficient 15-minute data
    try:
        insufficient_15m = tf15_data[:3]  # Only 3 candles, need 5
        signal_generator.generate_signal(tf5_data, insufficient_15m, 1.0850)
    except ValueError as e:
        print(f"✅ Caught insufficient 15m data error: {e}")

    # Example 3: Invalid risk management calculations
    try:
        # Test invalid signal type for risk calculations
        risk_mgmt.calculate_take_profit(1.0850, SignalType.NO_SIGNAL)
    except ValueError as e:
        print(f"✅ Caught invalid signal type error: {e}")

    # Example 4: Test graceful handling of missing divergence data
    print("\n🔍 Testing graceful degradation:")

    # With minimal data, divergence detection should return None gracefully
    minimal_5m = tf5_data[:6]  # Just enough for vector, not enough for divergence
    minimal_15m = tf15_data[:6]

    try:
        signal = signal_generator.generate_signal(minimal_5m, minimal_15m, 1.0850)
        print(f"✅ Signal generated with minimal data: {signal.signal_type.value}")
        print(f"   Divergence data available: {signal.divergence_data is not None}")
    except Exception as e:
        print(f"❌ Unexpected error with minimal data: {e}")

    print("\n🎯 Error handling demonstration completed")

demonstrate_signals_error_handling()

## 4. Real Data Integration (via DataService)

The real data functionality is seamlessly integrated into the DataService module via the tvkit library. This section demonstrates how to work with real-time data streaming and historical data fetching.

In [ ]:
async def demonstrate_real_data_integration() -> None:
    """
    Demonstrate real data integration capabilities.

    Note: This uses the DataService with tvkit for real data access.
    """
    print("📡 Demonstrating Real Data Integration")
    print("=" * 50)

    # Configuration for real data
    risk_mgmt: RiskManagement = RiskManagement(
        symbol="EURUSD",
        pip_size=0.0001,
        take_profit_pips=20,
        stop_loss_pips=30
    )

    config: StrategyConfig = StrategyConfig(
        symbol="EURUSD",
        exchange="FX_IDC",
        risk_management=risk_mgmt
    )

    try:
        async with DataService(config) as real_data_service:
            print("✅ Real data service initialized")

            # Test latest price fetching
            print("\n💰 Fetching latest price...")
            try:
                latest_price: float = await real_data_service.get_latest_price()
                print(f"   Latest {config.symbol} price: {latest_price:.5f}")
            except Exception as e:
                print(f"   ⚠️  Latest price fetch failed (using fallback): {e}")
                latest_price = tf5_data[-1].close
                print(f"   Fallback price: {latest_price:.5f}")

            # Test different timeframe data
            print("\n📊 Testing different timeframes...")
            timeframes: List[TimeFrame] = [TimeFrame.MIN_5, TimeFrame.MIN_15, TimeFrame.MIN_30]

            for tf in timeframes:
                try:
                    data: List[OHLCVData] = await real_data_service.fetch_historical_data(
                        timeframe=tf,
                        bars_count=10
                    )
                    print(f"   {tf.value}-minute: {len(data)} bars fetched")

                    if data:
                        latest_bar = data[-1]
                        print(f"     Latest bar: {latest_bar.datetime} | Close: {latest_bar.close:.5f}")

                except Exception as e:
                    print(f"   ❌ {tf.value}-minute fetch failed: {e}")

            # Demonstrate data validation
            print("\n🔍 Data validation demonstration:")
            sample_data = await real_data_service.fetch_historical_data(TimeFrame.MIN_5, 5)

            for i, bar in enumerate(sample_data[-3:], 1):
                print(f"   Bar {i}: Valid OHLCV = {type(bar).__name__}")
                print(f"     Timestamp: {bar.timestamp} -> {bar.datetime}")
                print(f"     OHLC: O={bar.open:.5f} H={bar.high:.5f} L={bar.low:.5f} C={bar.close:.5f}")
                print(f"     Volume: {bar.volume:.0f}")
                print(f"     True range: {bar.true_range:.5f}")

    except Exception as e:
        print(f"❌ Real data integration error: {e}")
        print("💡 Note: Real data requires internet connection and tvkit availability")

    print("\n🎯 Real data integration demonstration completed")

await demonstrate_real_data_integration()

## 5. Data Visualization

Let's create comprehensive visualizations to illustrate the vector scalping strategy components:

In [ ]:
def create_comprehensive_charts(
    df: pl.DataFrame,
    tf5_data: List[OHLCVData],
    trading_signal: TradingSignal
) -> None:
    """
    Create comprehensive charts for vector scalping strategy demonstration.

    Args:
        df: Polars DataFrame with technical indicators
        tf5_data: 5-minute OHLCV data
        trading_signal: Generated trading signal
    """
    print("📈 Creating Comprehensive Strategy Visualizations")
    print("=" * 50)

    # Convert DataFrame to pandas for matplotlib compatibility
    df_pandas = df.to_pandas()
    df_pandas['datetime'] = pd.to_datetime(df_pandas['timestamp'], unit='s')

    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Vector Scalping Strategy - Complete Analysis', fontsize=16, fontweight='bold')

    # Chart 1: Price Action with Moving Averages
    ax1 = axes[0, 0]
    ax1.plot(df_pandas['datetime'], df_pandas['close'], label='Close Price', linewidth=1.5, color='black')

    # Add moving averages if available
    if 'sma_5' in df_pandas.columns and df_pandas['sma_5'].notna().any():
        ax1.plot(df_pandas['datetime'], df_pandas['sma_5'], label='SMA(5)', linewidth=1, color='blue', alpha=0.7)
    if 'sma_20' in df_pandas.columns and df_pandas['sma_20'].notna().any():
        ax1.plot(df_pandas['datetime'], df_pandas['sma_20'], label='SMA(20)', linewidth=1, color='red', alpha=0.7)

    # Mark signal entry point if available
    if trading_signal.signal_type != SignalType.NO_SIGNAL and trading_signal.entry_price:
        signal_time = df_pandas['datetime'].iloc[-1]  # Latest timestamp
        color = 'green' if trading_signal.signal_type == SignalType.LONG else 'red'
        marker = '^' if trading_signal.signal_type == SignalType.LONG else 'v'
        ax1.scatter([signal_time], [trading_signal.entry_price],
                   color=color, marker=marker, s=100, zorder=5,
                   label=f'{trading_signal.signal_type.value} Signal')

        # Add TP/SL lines
        if trading_signal.take_profit:
            ax1.axhline(y=trading_signal.take_profit, color='green', linestyle='--', alpha=0.6, label='Take Profit')
        if trading_signal.stop_loss:
            ax1.axhline(y=trading_signal.stop_loss, color='red', linestyle='--', alpha=0.6, label='Stop Loss')

    ax1.set_title('Price Action & Signal Analysis')
    ax1.set_ylabel('Price')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Chart 2: Volume and ATR
    ax2 = axes[0, 1]
    ax2_twin = ax2.twinx()

    # Volume bars
    ax2.bar(df_pandas['datetime'], df_pandas['volume'], alpha=0.6, color='lightblue', label='Volume')
    ax2.set_ylabel('Volume', color='blue')
    ax2.tick_params(axis='y', labelcolor='blue')

    # ATR line
    if 'atr_5' in df_pandas.columns and df_pandas['atr_5'].notna().any():
        ax2_twin.plot(df_pandas['datetime'], df_pandas['atr_5'], color='orange', linewidth=2, label='ATR(5)')
        ax2_twin.set_ylabel('ATR', color='orange')
        ax2_twin.tick_params(axis='y', labelcolor='orange')

    ax2.set_title('Volume & Average True Range')
    ax2.grid(True, alpha=0.3)

    # Chart 3: Vector Analysis
    ax3 = axes[1, 0]

    # Calculate rolling vector magnitudes for visualization
    vector_magnitudes = []
    vector_directions = []
    timestamps = []

    period = 5
    for i in range(period, len(tf5_data)):
        window = tf5_data[i-period:i]
        try:
            momentum_vec = VectorCalculations.calculate_momentum_vector(window, period)
            vector_magnitudes.append(momentum_vec.magnitude)
            vector_directions.append(momentum_vec.direction)
            timestamps.append(datetime.fromtimestamp(window[-1].timestamp))
        except:
            continue

    if vector_magnitudes:
        ax3.plot(timestamps, vector_magnitudes, label='Vector Magnitude', color='purple', linewidth=2)
        ax3_twin = ax3.twinx()
        ax3_twin.plot(timestamps, vector_directions, label='Vector Direction', color='brown', linewidth=1.5)
        ax3_twin.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
        ax3_twin.axhline(y=0.3, color='green', linestyle='--', alpha=0.5, label='Long Threshold')
        ax3_twin.axhline(y=-0.3, color='red', linestyle='--', alpha=0.5, label='Short Threshold')

        ax3.set_ylabel('Magnitude', color='purple')
        ax3_twin.set_ylabel('Direction', color='brown')
        ax3.tick_params(axis='y', labelcolor='purple')
        ax3_twin.tick_params(axis='y', labelcolor='brown')
        ax3_twin.legend(loc='upper right')

    ax3.set_title('Vector Analysis (Momentum)')
    ax3.legend(loc='upper left')
    ax3.grid(True, alpha=0.3)

    # Chart 4: Signal Strength and Percentile Ranking
    ax4 = axes[1, 1]

    # Create signal strength visualization
    if vector_magnitudes:
        # Calculate percentile rankings
        percentile_ranks = []
        for i, magnitude in enumerate(vector_magnitudes):
            if i >= 10:  # Need historical data
                historical = vector_magnitudes[:i]
                rank = VectorCalculations.calculate_percentile_rank(magnitude, historical)
                percentile_ranks.append(rank)
            else:
                percentile_ranks.append(50.0)  # Default

        # Plot signal strength
        valid_timestamps = timestamps[:len(percentile_ranks)]
        ax4.plot(valid_timestamps, percentile_ranks, label='Signal Strength', color='navy', linewidth=2)
        ax4.axhline(y=60, color='green', linestyle='--', alpha=0.7, label='Signal Threshold (60th)')
        ax4.axhline(y=80, color='orange', linestyle='--', alpha=0.7, label='Strong Signal (80th)')
        ax4.fill_between(valid_timestamps, 60, 100, alpha=0.1, color='green', label='Signal Zone')

        # Mark current signal strength
        if trading_signal.vector_data:
            current_strength = trading_signal.vector_data.signal_strength
            ax4.scatter([valid_timestamps[-1]], [current_strength],
                       color='red', s=100, zorder=5, label=f'Current: {current_strength:.1f}th')

    ax4.set_title('Signal Strength (Percentile Ranking)')
    ax4.set_ylabel('Percentile')
    ax4.set_ylim(0, 100)
    ax4.legend()
    ax4.grid(True, alpha=0.3)

    # Format x-axis for all subplots
    for ax in axes.flat:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

    plt.tight_layout()
    plt.show()

    print("📊 Charts created successfully")
    print(f"   📈 Price chart with {len(df_pandas)} data points")
    print(f"   📊 Vector analysis with {len(vector_magnitudes)} calculations")
    print(f"   🎯 Current signal: {trading_signal.signal_type.value}")

# Import pandas for matplotlib compatibility
import pandas as pd

# Create comprehensive visualizations
create_comprehensive_charts(df_with_indicators, tf5_data, trading_signal)

## 6. Complete Strategy Workflow Example

Finally, let's demonstrate a complete end-to-end workflow that combines all modules:

In [ ]:
async def complete_strategy_workflow() -> None:
    """
    Demonstrate complete end-to-end vector scalping strategy workflow.

    This function showcases the integration of all modules:
    - DataService for real-time data
    - VectorCalculations for mathematical analysis
    - SignalGenerator for trading decisions
    - Comprehensive error handling throughout
    """
    print("🚀 Complete Vector Scalping Strategy Workflow")
    print("=" * 60)

    try:
        # Step 1: Configuration Setup
        print("\n1️⃣ Strategy Configuration")
        risk_mgmt: RiskManagement = RiskManagement(
            symbol="EURUSD",
            pip_size=0.0001,
            take_profit_pips=20,
            stop_loss_pips=30,
            is_decimal_4=True
        )

        config: StrategyConfig = StrategyConfig(
            symbol="EURUSD",
            exchange="FX_IDC",
            vector_period=5,
            percentile_lookback=100,
            signal_threshold=60.0,
            direction_threshold=0.3,
            tf5_weight=0.7,
            tf15_weight=0.3,
            tf5_direction_weight=0.6,
            tf15_direction_weight=0.4,
            risk_management=risk_mgmt
        )
        print(f"✅ Configured for {config.symbol} with {config.vector_period}-candle vectors")

        # Step 2: Data Acquisition
        print("\n2️⃣ Data Acquisition")
        async with DataService(config) as data_service:
            # Fetch multi-timeframe data
            market_data: Dict[TimeFrame, List[OHLCVData]] = await data_service.fetch_multi_timeframe_data(50)
            tf5_data = market_data[TimeFrame.MIN_5]
            tf15_data = market_data[TimeFrame.MIN_15]
            print(f"📊 Fetched {len(tf5_data)} 5-min bars and {len(tf15_data)} 15-min bars")

            # Get current market price
            try:
                current_price: float = await data_service.get_latest_price()
            except Exception:
                current_price = tf5_data[-1].close
            print(f"💰 Current price: {current_price:.5f}")

            # Step 3: Vector Analysis
            print("\n3️⃣ Vector Analysis")

            # Calculate individual vectors
            tf5_price: PriceVector = VectorCalculations.calculate_price_vector(tf5_data, config.vector_period)
            tf5_momentum: MomentumVector = VectorCalculations.calculate_momentum_vector(tf5_data, config.vector_period)
            tf15_price: PriceVector = VectorCalculations.calculate_price_vector(tf15_data, config.vector_period)
            tf15_momentum: MomentumVector = VectorCalculations.calculate_momentum_vector(tf15_data, config.vector_period)

            # Combine vectors
            combined: CombinedVector = VectorCalculations.combine_vectors(
                tf5_price, tf5_momentum, tf15_price, tf15_momentum,
                config.tf5_weight, config.tf15_weight,
                config.tf5_direction_weight, config.tf15_direction_weight
            )

            print(f"🧮 5m vectors: direction={tf5_momentum.direction:.3f}, magnitude={tf5_momentum.magnitude:.5f}")
            print(f"🧮 15m vectors: direction={tf15_momentum.direction:.3f}, magnitude={tf15_momentum.magnitude:.5f}")
            print(f"🔄 Combined: direction={combined.combined_direction:.3f}, magnitude={combined.combined_magnitude:.5f}")

            # Step 4: Signal Generation
            print("\n4️⃣ Signal Generation")
            signal_generator: SignalGenerator = SignalGenerator(config)

            # Build historical data for percentile calculation
            for i in range(20):
                if len(tf5_data) > i + config.vector_period and len(tf15_data) > i + config.vector_period:
                    end_idx = -i if i > 0 else None
                    window_5m = tf5_data[-(config.vector_period + i):end_idx]
                    window_15m = tf15_data[-(config.vector_period + i):end_idx]

                    if len(window_5m) >= config.vector_period and len(window_15m) >= config.vector_period:
                        temp_price = window_5m[-1].close
                        temp_signal = signal_generator.generate_signal(window_5m, window_15m, temp_price)

            # Generate final trading signal
            signal: TradingSignal = signal_generator.generate_signal(tf5_data, tf15_data, current_price)

            print(f"🎯 Signal: {signal.signal_type.value} (confidence: {signal.confidence:.2f})")
            print(f"📊 Signal strength: {signal.vector_data.signal_strength:.1f}th percentile")
            print(f"📝 Reason: {signal.reason}")

            # Step 5: Risk Management
            print("\n5️⃣ Risk Management")
            if signal.signal_type != SignalType.NO_SIGNAL:
                print("💼 Trade Setup:")
                print(f"   Entry: {signal.entry_price:.5f}")
                print(f"   Take Profit: {signal.take_profit:.5f}")
                print(f"   Stop Loss: {signal.stop_loss:.5f}")

                # Calculate risk metrics
                if signal.signal_type == SignalType.LONG:
                    risk_pips = (signal.entry_price - signal.stop_loss) / config.risk_management.pip_size
                    reward_pips = (signal.take_profit - signal.entry_price) / config.risk_management.pip_size
                else:
                    risk_pips = (signal.stop_loss - signal.entry_price) / config.risk_management.pip_size
                    reward_pips = (signal.entry_price - signal.take_profit) / config.risk_management.pip_size

                risk_reward_ratio = reward_pips / risk_pips if risk_pips > 0 else 0
                print(f"   Risk: {risk_pips:.0f} pips | Reward: {reward_pips:.0f} pips")
                print(f"   Risk/Reward: 1:{risk_reward_ratio:.2f}")
            else:
                print("⚪ No trade signal - conditions not met")

            # Step 6: Additional Analysis
            print("\n6️⃣ Additional Analysis")

            # Check time-based exit
            should_exit_time = signal_generator.should_exit_time_based()
            print(f"⏰ Time-based exit (Friday 5 PM GMT): {should_exit_time}")

            # Divergence analysis
            if signal.divergence_data:
                div = signal.divergence_data
                print(f"🔄 Divergence detected: Bullish={div.is_bullish_divergence}, Bearish={div.is_bearish_divergence}")
                print(f"   Strength: {div.divergence_strength:.3f}")
            else:
                print("🔄 No divergence data available (insufficient history)")

            # Data quality metrics
            print("\n📊 Data Quality:")
            print(f"   5m data span: {tf5_data[0].datetime} to {tf5_data[-1].datetime}")
            print(f"   15m data span: {tf15_data[0].datetime} to {tf15_data[-1].datetime}")
            print(f"   Historical magnitude points: {len(signal_generator.historical_magnitudes)}")

            # Step 7: Summary
            print("\n🎯 Strategy Summary")
            print("=" * 30)
            if signal.signal_type != SignalType.NO_SIGNAL:
                confidence_level = "High" if signal.confidence > 0.7 else "Medium" if signal.confidence > 0.5 else "Low"
                print(f"🚦 ACTIONABLE SIGNAL: {signal.signal_type.value}")
                print(f"🎲 Confidence Level: {confidence_level} ({signal.confidence:.2f})")
                print(f"⚡ Signal Strength: {signal.vector_data.signal_strength:.1f}th percentile")
                print(f"📈 Direction Bias: {signal.vector_data.combined_direction:.3f}")
            else:
                print("⚪ NO SIGNAL - Market conditions not favorable")
                print(f"📊 Current strength: {signal.vector_data.signal_strength:.1f}th percentile")
                print(f"🎯 Required threshold: {config.signal_threshold}th percentile")

            print("\n✅ Complete workflow executed successfully!")

    except Exception as e:
        print(f"❌ Workflow error: {e}")
        import traceback
        traceback.print_exc()
        raise

# Execute complete workflow
await complete_strategy_workflow()

## Summary

This notebook has demonstrated the comprehensive usage of all vector scalping strategy modules:

### 📊 **DataService Module**
- **Async data fetching** using tvkit library for real-time forex data
- **Multi-timeframe support** (5-min, 15-min, etc.)
- **Polars integration** for efficient data analysis
- **Fallback mechanisms** with mock data when real data unavailable
- **Type safety** with Pydantic validation

### 🧮 **Calculations Module** 
- **Price Vector calculation**: displacement, magnitude, direction over N candles
- **Momentum Vector calculation**: price momentum + volatility (ATR)
- **Multi-timeframe combination**: weighted averaging of 5m and 15m vectors
- **Percentile ranking**: signal strength calculation
- **Divergence detection**: price vs momentum analysis

### 🎯 **Signals Module**
- **Trading signal generation**: LONG/SHORT conditions based on vector analysis
- **Confidence scoring**: multi-factor confidence calculation
- **Risk management integration**: automatic TP/SL calculation
- **Time-based exits**: Friday 5 PM GMT logic
- **Historical data management**: rolling percentile windows

### 📡 **Real Data Integration**
- **Live data streaming** capabilities via tvkit
- **Historical data fetching** with multiple timeframes
- **Data validation** through Pydantic models
- **Error resilience** with graceful fallbacks

### 🔒 **Error Handling**
All modules demonstrate robust error handling:
- **Input validation** with meaningful error messages
- **Graceful degradation** when data unavailable
- **Type safety** enforcement throughout
- **Fallback mechanisms** for production reliability

### 📈 **Visualizations**
- **Price action charts** with moving averages and signals
- **Volume and ATR analysis**
- **Vector magnitude and direction plots**
- **Signal strength percentile tracking**

The vector scalping strategy follows all architectural standards:
- ✅ **Async-first architecture** for all I/O operations
- ✅ **Type annotations** throughout for IDE support and runtime safety
- ✅ **Pydantic models** for data validation and serialization
- ✅ **Comprehensive error handling** with informative messages
- ✅ **Production-ready code** with logging and monitoring

This notebook serves as both documentation and a working example of how to integrate all modules for a complete trading strategy implementation.